In [1]:
!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fileId = drive.CreateFile({'id': '1fbvd-FVNUvU8xf6hpwYzpBwENpQTPVfl'}) 
print(fileId['title'])  # dataset.zip
fileId.GetContentFile('temp.zip')  # Save Drive file as a local file

!unzip temp.zip -d ./

imdb_master.zip
Archive:  temp.zip
replace ./imdb_master.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./imdb_master.csv       


In [2]:
# GloVe file

fileId = drive.CreateFile({'id': '1qOlzqoAmmdfvUu8Tr9rujwA5_L8_ikTl'}) 
print(fileId['title'])  # dataset.zip
fileId.GetContentFile('glove6b100dtxt.zip')  # Save Drive file as a local file

!unzip glove6b100dtxt.zip -d ./


glove6b100dtxt.zip
Archive:  glove6b100dtxt.zip
  inflating: ./glove.6B.100d.txt     


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
df = pd.read_csv('imdb_master.csv', encoding = "ISO-8859-1")
del df['Unnamed: 0']
del df['file']
del df['type']

In [0]:
df = df.loc[0:49999]
df = df.sample(frac=1).reset_index(drop=True)

In [0]:
def clean_reviews(text):
    lemmatizer = WordNetLemmatizer()
    my_stopwords = stopwords.words('english') 
    text = text.replace("<br >", "")
    text = text.replace("</br >", "")        
    text = re.sub('[^a-zA-Z]',' ', text)
    text = text.lower() 
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in my_stopwords]
    text = " ".join(text)
    return text  

In [0]:
df["review"]=df.review.apply(lambda x: clean_reviews(x))

In [8]:
df.head()

,review,label
0,privilege see powerful play broadway kathy bat...,neg
1,amaze many people know see little gem everybo...,pos
2,probably could skip begin sure start asian p...,pos
3,movie seem fall two category film reinforce e...,pos
4,charge feather river routine western u cav...,pos


In [9]:
df["label"] = df["label"].map({'pos': 1, 'neg': 0})
df.head()

,review,label
0,privilege see powerful play broadway kathy bat...,0
1,amaze many people know see little gem everybo...,1
2,probably could skip begin sure start asian p...,1
3,movie seem fall two category film reinforce e...,1
4,charge feather river routine western u cav...,1


In [0]:
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Bidirectional
from keras.layers.embeddings import Embedding
import string

Using TensorFlow backend.


In [0]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['review'])
sequences = tokenizer.texts_to_sequences(df['review'])
data = pad_sequences(sequences, maxlen=200)


embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [13]:
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 100, input_length=200, weights=[embedding_matrix], trainable=False))
model_glove.add(LSTM(500, dropout=0.2, recurrent_dropout=0.2))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_glove.fit(data, df['label'], validation_split=0.2, epochs = 10)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 1037s 26ms/step - loss: 0.5072 - acc: 0.7499 - val_loss: 0.3622 - val_acc: 0.8406
Epoch 2/10
40000/40000 [==============================] - 1022s 26ms/step - loss: 0.3714 - acc: 0.8351 - val_loss: 0.3274 - val_acc: 0.8576
Epoch 3/10
40000/40000 [==============================] - 1005s 25ms/step - loss: 0.3368 - acc: 0.8531 - val_loss: 0.3226 - val_acc: 0.8614
Epoch 4/10
40000/40000 [==============================] - 1007s 25ms/step - loss: 0.3095 - acc: 0.8667 - val_loss: 0.3017 - val_acc: 0.8695
Epoch 5/10
40000/40000 [==============================] - 1020s 25ms/step - loss: 0.2822 - acc: 0.8804 - val_loss: 0.2900 - val_acc: 0.8775
Epoch 6/10
4000

In [14]:
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 100, input_length=200, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(256, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(Bidirectional(LSTM(500, dropout=0.2, recurrent_dropout=0.2)))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_glove.fit(data, df['label'], validation_split=0.2, epochs = 10)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 481s 12ms/step - loss: 0.4513 - acc: 0.7839 - val_loss: 0.3608 - val_acc: 0.8374
Epoch 2/10
40000/40000 [==============================] - 497s 12ms/step - loss: 0.3555 - acc: 0.8428 - val_loss: 0.3304 - val_acc: 0.8547
Epoch 3/10
40000/40000 [==============================] - 494s 12ms/step - loss: 0.3200 - acc: 0.8631 - val_loss: 0.3381 - val_acc: 0.8601
Epoch 4/10
40000/40000 [==============================] - 494s 12ms/step - loss: 0.2914 - acc: 0.8755 - val_loss: 0.3051 - val_acc: 0.8704
Epoch 5/10
40000/40000 [==============================] - 497s 12ms/step - loss: 0.2598 - acc: 0.8909 - val_loss: 0.3131 - val_acc: 0.8672
Epoch 6/10
40000/40000 [==============================] - 498s 12ms/step - loss: 0.2356 - acc: 0.9042 - val_loss: 0.3202 - val_acc: 0.8692
Epoch 7/10
40000/40000 [==============================] - 494s 12ms/step - loss: 0.2151 - acc: 0.9107 - val_loss: 0.